# Model Evaluation

Loads the pre-processed Chinook data, trains multiple models, evaluates them, and logs every run to MLflow (parameters, metrics, artifacts, and the serialized models)

## 1. Setup & Imports

In [1]:
%pip install -q mlflow scikit-learn pandas matplotlib seaborn joblib

import mlflow, mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import joblib, os, json

Note: you may need to restart the kernel to use updated packages.


## 2. Set MLflow Experiment

In [2]:
experiment_name = "Chinook-High-Spender"
mlflow.set_experiment(experiment_name)
# Turn on autologging for scikit-learn
mlflow.sklearn.autolog(log_models=True)
print(f"Active MLflow experiment: {experiment_name}")

2025/07/16 12:02:53 INFO mlflow.tracking.fluent: Experiment with name 'Chinook-High-Spender' does not exist. Creating a new experiment.


Active MLflow experiment: Chinook-High-Spender


## 3. Load Training & Test Data

In [3]:
train_path = 'C:/Users/milan/Desktop/chinook/train_data.csv'
test_path = 'C:/Users/milan/Desktop/chinook/test_data.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(f'Train shape: {train_df.shape} \
Test shape : {test_df.shape}')

X_train = train_df.drop(columns=['target'])
y_train = train_df['target']
X_test  = test_df.drop(columns=['target'])
y_test  = test_df['target']

Train shape: (47, 12) Test shape : (12, 12)


## 4. Define Candidate Models

In [4]:
models = {
    'RandomForest': RandomForestClassifier(n_estimators=200, random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=500, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

## 5. Train, Evaluate & Log with MLflow

In [5]:
results = []
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # ----- Train -----
        model.fit(X_train, y_train)

        # ----- Predict -----
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)
        mlflow.log_metric('test_accuracy', acc)

        # Confusion-matrix plot as artifact
        fig, ax = plt.subplots(figsize=(8, 6))  # Create new figure and axes
        ConfusionMatrixDisplay.from_predictions(
            y_test, 
            preds,
            ax=ax,
            cmap='Blues',
            display_labels=['Regular', 'High Spender']
        )
        ax.set_title(f'{name} Confusion Matrix')
        plt.tight_layout()
        
        # Save and log confusion matrix
        fig_path = f'{name}_cm.png'
        fig.savefig(fig_path, bbox_inches='tight', dpi=300)
        plt.close(fig)  # Close figure to free memory
        mlflow.log_artifact(fig_path)
        os.remove(fig_path)  # Clean up temporary file

        # Keep record for summary
        results.append({
            'model': name, 
            'test_accuracy': acc, 
            'run_id': mlflow.active_run().info.run_id
        })

print("Logged runs:")
print(pd.DataFrame(results))

2025/07/16 12:02:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\milan\Desktop\chinook\myenv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/16 12:02:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\milan\Desktop\chinook\myenv\Lib\si

Logged runs:
                model  test_accuracy                            run_id
0        RandomForest       0.916667  039a8cd57546423aa5e74937f6f5b0bd
1  LogisticRegression       1.000000  66aff92db7e0494a8ebd9c731afbee7a
2                 SVM       0.750000  12b380ebc15a406b9d44f4c24a65161e


## 7. MLflow UI


To open the MLflow tracking UI locally, run the following command in your terminal from the project root:

```bash
mlflow ui --port 5000
```

Then visit [http://127.0.0.1:5000](http://127.0.0.1:5000) in your browser to explore runs, metrics, and artifacts.
